# Permutation of graphs

The following notebook demonstrates the usage of permutation_utils.py to permute bitstrings, graphs and histograms. Special emphasis is put on the illustration of the connection of $\tau = \sigma \circ \rho$. 

In [1]:
%cd ../
%matplotlib inline
import numpy as np
import scipy.sparse as sps

from core.asym_verification_protocol import permutation_utils
from core.audio_cwe import watermarking_utils

/Users/gru/Documents/skripte/CSM/scripts/ws_15_16/38777 Masterthesis/src/audio_cwe_framework


## Permute a graph
The permutation of a graph is achieved by shuffling its nodes/vertices. To achieve a structurally equal graph the edges have to be set accordingly. If and only if two nodes of the original graph $u, v \in V(G_1)$ are adjacent, then $\varphi(u), \varphi(v)$ have to share an edge in $G_2$, too.

In [2]:
# %load -s permute_graph ./core/asym_verification_protocol/permutation_utils.py
def permute_graph(graph, permutation):
    """Shuffles the nodes of a given graph according to a given permutation
    and forms the resulting adjacency matrix.

    :param graph: a sparse adjacency matrix (in csr format), which represents
    the graph
    :param permutation: the permutation to apply
    :return: shuffled_graph: the resulting graph's adjacency matrix in
    (csr format)
    """
    nnz_i, nnz_j = graph.nonzero()
    graph_adjacency = graph.toarray()
    shuffled_adjacency = np.zeros(
        (graph_adjacency.shape[0], graph_adjacency.shape[1]))

    for x in range(len(nnz_i)):
        new_i = permutation[nnz_i[x]]
        new_j = permutation[nnz_j[x]]

        shuffled_adjacency[new_i][new_j] = graph_adjacency[nnz_i[x]][nnz_j[x]]

    shuffled_graph = sps.csr_matrix(shuffled_adjacency)
    return shuffled_graph


In [3]:
# Generate random graph with 5 nodes
size = 5
graph = sps.rand(size, size, density=0.3, format='csr')
print('Original graph:')
print(graph)

p1 = permutation_utils.generate_random_permutation(size)
print('Applied permutation:\n', p1)
graph_shuffled = permute_graph(graph, p1) 
print('Shuffled graph:')
print(graph_shuffled)

Original graph:
  (0, 0)	0.47863980862
  (1, 1)	0.473807696921
  (3, 0)	0.208383566855
  (3, 2)	0.445743586384
  (3, 3)	0.988946822035
  (3, 4)	0.0831054439091
  (4, 2)	0.639887875999
Applied permutation:
 [3 4 1 0 2]
Shuffled graph:
  (0, 0)	0.988946822035
  (0, 1)	0.445743586384
  (0, 2)	0.0831054439091
  (0, 3)	0.208383566855
  (2, 1)	0.639887875999
  (3, 3)	0.47863980862
  (4, 4)	0.473807696921


## Inversion of a permutation

In [4]:
# %load -s invert ./core/asym_verification_protocol/permutation_utils.py
def invert(permutation):
    """Inverts a given permutation

    :param permutation: the permutation to be inverted
    :return: inv_perm: the inverted permutation
    """
    inv_perm = np.zeros(len(permutation), dtype=permutation[0].dtype)

    for i, v in enumerate(permutation):
        inv_perm[v] = i
    return inv_perm


In [5]:
# Generate random graph with 100 nodes
size = 100
graph = sps.rand(size, size, density=0.2, format='csr')

# Generate a permutation by the use of a logistic map with the following params
x_n = 0.7
mu = 3.7
i = 100

tau = permutation_utils.generate_permutation(size, mu, x_n, i)
print("Tau:\n",tau)

# Invert the permutation
inv_tau = invert(tau)
print("Inverse Tau:\n", inv_tau)

# Apply permutation
shuffled_graph = permutation_utils.permute_graph(graph, tau)

# Apply inverse permutation
inv_shuffled_graph = permutation_utils.permute_graph(shuffled_graph, inv_tau)

# Compare original graph and the resulting one
if np.array_equal(graph.toarray(), inv_shuffled_graph.toarray()):
    print("Original graph == shuffled and then inverted graph")

Tau:
 [69 91 16 58 76 98 23 38 52 32  2 65 87 12 48 83  8  6 10 85 30 50 36 56 14
 89 67 74 96 21  0 63 46 81  4 28 34 54 72 94 19 61 44 79 26 70 92 17 59 42
 77 99 24 40 39 41 25 78 43 60 18 93 71 53 33 27  3 80 45 62 20 95 73 66 88
 13 55 35 49 29 84  9  5  7 82 47 11 86 64  1 31 51 37 22 97 75 57 15 90 68]
Inverse Tau:
 [30 89 10 66 34 82 17 83 16 81 18 86 13 75 24 97  2 47 60 40 70 29 93  6 52
 56 44 65 35 79 20 90  9 64 36 77 22 92  7 54 53 55 49 58 42 68 32 85 14 78
 21 91  8 63 37 76 23 96  3 48 59 41 69 31 88 11 73 26 99  0 45 62 38 72 27
 95  4 50 57 43 67 33 84 15 80 19 87 12 74 25 98  1 46 61 39 71 28 94  5 51]
Original graph == shuffled and then inverted graph


## Permute graph $g$
The following cell illustrates the coherence of $\tau$, $\rho$ and $\sigma$ and shows, that $\rho(g) = \sigma^{-1}(\tau(g))$ holds.

In [6]:
# Generate random graph with 5 nodes (for testability by hand)
size = 5
graph = sps.rand(size, size, density=0.2, format='csr')

# Generate a arbitrary permutation by the use of a logistic map with the following params
x_n = 0.4332
mu = 3.6987
i = 500

tau = permutation_utils.generate_permutation(size, mu, x_n, i)
print("Tau:\n", tau)

# Generate another arbitrary permutation
x_n = 0.123
mu = 3.64
i = 500

rho = permutation_utils.generate_permutation(size, mu, x_n, i)
print("Rho:\n", rho)

# Construct a permutation \sigma, so that \sigma \circ \rho==\tau
sigma = permutation_utils.partial_permutation(rho, tau)
print("Sigma:\n", sigma)
# Invert \sigma
inv_sigma = permutation_utils.invert(sigma)

# Permute the graph 
tau_graph = permutation_utils.permute_graph(graph, tau)
rho_graph= permutation_utils.permute_graph(graph, rho)
inv_sigma_tau_graph = permutation_utils.permute_graph(tau_graph, inv_sigma)

# Compare the results
if np.array_equal(rho_graph.toarray(), inv_sigma_tau_graph.toarray()):
    print("rho(g) == sigma^{-1}(tau(g))")

Tau:
 [3 1 0 4 2]
Rho:
 [4 0 2 1 3]
Sigma:
 [1 4 0 2 3]
rho(g) == sigma^{-1}(tau(g))


## Permute a bitstring $m$
The following piece of code shows, that $\tau(m) == \sigma(\rho(m))$ is true.

In [7]:
mark = watermarking_utils.construct_watermark('HdM')

# Generate \tau
tau = permutation_utils.generate_random_permutation(len(mark))
print("Tau:\n", tau)

# Construct another permutation, this time pseudo-randomly
rho = permutation_utils.generate_random_permutation(len(mark))
print("Rho:\n", rho)

# Construct sigma, so that \sigma \circ \rho = \tau
sigma = permutation_utils.partial_permutation(rho, tau)
print("Sigma:\n", sigma)

mark_a = permutation_utils.permute_list(mark, tau)
mark_b = permutation_utils.permute_list(mark, rho)
mark_c = permutation_utils.permute_list(mark_b, sigma)

if np.array_equal(mark_a, mark_c):
    print('\\tau(m) == \\sigma(\\rho(m))')

Tau:
 [23 20 10 13  5  0 16 18 21  8 11  3 14  6  1  2  7 17 15  4 12  9 19 22]
Rho:
 [17 11  7  5 15  9  3 13 23  1 21 19 18 20  0 22 12  2  8 14  4  6 10 16]
Sigma:
 [ 1  8 17 16 12 13  9 10 15  0 19 20  7 18  4  5 22 23 14  3  6 11  2 21]
\tau(m) == \sigma(\rho(m))
